# Fine-Tuning Llama OpenThaiGPT

Credit: [younesbelkada](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da)

## Install Libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import Libraries

In [2]:
import os
import torch
from datasets import load_dataset, Dataset,DatasetDict
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import sentencepiece
import pandas as pd
from sklearn.model_selection import train_test_split

## Model Setting

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "openthaigpt/openthaigpt-1.0.0-alpha-7b-chat-ckpt-hf"

# Fine-tuned model name
new_model = "openthaigpt-alpha-7b-thai-med-pack"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 1e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

## Dataset

In [4]:
# Load Dataset from Huggingface Hub

# The instruction dataset to use
dataset_name = "Thaweewat/chatmed-5k-th" #"mlabonne/guanaco-llama2-1k"

# dataset for train
dataset = load_dataset(dataset_name, split="train")
dataset

Dataset({
    features: ['instruction', 'output', 'input'],
    num_rows: 5452
})

In [5]:
dataset = dataset.map(lambda example: {
    'text': f"<s>[INST] {example['instruction']} [INST] {example['output']} </s>",
})
dataset[0]

{'instruction': ' ฉันประสบกับอาการตื่นตระหนกกะทันหันและบ่อยครั้ง ฉันไม่รู้จะทำยังไงดี',
 'output': 'ก็ ตามที่คุณบอกมา ดูเหมือนว่าคุณอาจจะเป็นโรคตื่นตระหนก แนวทางปฏิบัติที่ดีที่สุดคือเริ่มด้วยการทำจิตบำบัดและการให้คำปรึกษาด้านสุขภาพจิต นอกจากนี้ เราควรทำการตรวจคลื่นไฟฟ้าหัวใจเพื่อให้แน่ใจว่าไม่มีปัญหาทางกายภาพที่ทำให้คุณมีอาการตื่นตระหนก เรายังต้องทำการตรวจคัดกรองภาวะซึมเศร้าและการตรวจพิษวิทยาเพื่อแยกแยะสาเหตุอื่นๆ สุดท้ายนี้ ฉันขอแนะนำการประเมินและการบำบัดทางจิตวิทยาและจิตเวชที่ครอบคลุมเพื่อช่วยจัดการกับอาการของคุณ',
 'input': '',
 'text': '<s>[INST]  ฉันประสบกับอาการตื่นตระหนกกะทันหันและบ่อยครั้ง ฉันไม่รู้จะทำยังไงดี [INST] ก็ ตามที่คุณบอกมา ดูเหมือนว่าคุณอาจจะเป็นโรคตื่นตระหนก แนวทางปฏิบัติที่ดีที่สุดคือเริ่มด้วยการทำจิตบำบัดและการให้คำปรึกษาด้านสุขภาพจิต นอกจากนี้ เราควรทำการตรวจคลื่นไฟฟ้าหัวใจเพื่อให้แน่ใจว่าไม่มีปัญหาทางกายภาพที่ทำให้คุณมีอาการตื่นตระหนก เรายังต้องทำการตรวจคัดกรองภาวะซึมเศร้าและการตรวจพิษวิทยาเพื่อแยกแยะสาเหตุอื่นๆ สุดท้ายนี้ ฉันขอแนะนำการประเมินและการบำบัดทางจิตว

In [ ]:
# # Load Dataset from Local Files

# df = pd.read_csv("path/to/your/datasets.csv")

# # Split train dataset
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# # Convert to DatasetDict
# train_dataset_dict = DatasetDict({
#     "train": Dataset.from_pandas(train_df),
# })

# # dataset for train
# dataset = train_dataset_dict['train']

# Fine-Tuning

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = LlamaForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text", #  Name of the field in the dataset that contains the text.
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/39 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5452 [00:00<?, ? examples/s]

Step,Training Loss
25,0.952400
50,1.246900
75,0.888700
100,1.117200
125,0.843400
150,0.997600
175,0.786900
200,0.924800
225,0.793400
250,0.847700


In [8]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "ปวดหัวมาก เป็นมาสองวันแล้ว"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ปวดหัวมาก เป็นมาสองวันแล้ว [/INST] จากอาการของคุณ ดูเหมือนว่าคุณอาจเป็นโรคเมนิโกรอยด์ ซึ่งเป็นโรคที่เกิดจากการอักเสบของเนื้อเยื่อในสมอง ซึ่งอาจทำให้เกิดอาการปวดหัว ปวดศีรษะ และปวดหลัง เรา
